<a href="https://colab.research.google.com/github/freedomtan/clip_score_on_android/blob/main/verify_text_encoder_fp32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q diffusers
!pip install -q accelerate
!pip install -q keras_cv

In [2]:
from diffusers import StableDiffusionPipeline
import torch

import keras_cv
import keras

model_id = "runwayml/stable-diffusion-v1-5"
# pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to("cuda")

torch_text_encoder = pipe.text_encoder
# keras.mixed_precision.set_global_policy("mixed_float16")

pipeline = keras_cv.models.StableDiffusion(
    img_width=512, img_height=512, jit_compile=True
)

Using TensorFlow backend


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [3]:
first_10_prompts = [
    "A city at night with people walking around.",
    "The large clock was prominently displaying the time.",
    "A person in winter gear riding a snowboard.",
    "A small bird is perched on an empty bird feeder.",
    "Baseballs players sliding to base and jumping during the game.",
    "A woman playing tennis in a white outfit",
    "A woman looking at her phone next to a car being towed.",
    "A man in a wheelchair and another sitting on a bench that is overlooking the water.",
    "Rows of motor bikes and helmets in a city"
]

In [4]:
for p in first_10_prompts:
  ids = pipe.tokenizer.encode(p)
  id_len = len(ids)
  ids_padded = ids + ([49407] * (77 - id_len))

  token = torch.unsqueeze(torch.tensor(ids_padded).to("cuda"), 0)
  torch_output = torch_text_encoder(input_ids=token).last_hidden_state.cpu().detach().numpy()
  tf_output = pipeline.encode_text(p)
  print(torch_output - tf_output)

[[[ 0.0000000e+00  2.4214387e-08  3.3527613e-08 ... -2.9802322e-08
    2.9802322e-08  1.4901161e-08]
  [ 5.2154064e-08  9.5367432e-07  8.9406967e-08 ...  2.9802322e-07
    1.1920929e-07 -5.9604645e-08]
  [ 2.3841858e-07 -5.9604645e-08 -2.3841858e-07 ...  0.0000000e+00
    0.0000000e+00 -1.0728836e-06]
  ...
  [-8.9406967e-08 -3.5762787e-07  5.2154064e-07 ...  2.3841858e-07
    1.4901161e-07  2.3841858e-07]
  [ 0.0000000e+00 -1.1920929e-07 -6.7055225e-07 ...  5.9604645e-07
    2.3841858e-07  5.9604645e-08]
  [ 5.9604645e-08 -2.3841858e-07 -5.9604645e-08 ... -1.7881393e-07
   -1.4901161e-07 -1.7881393e-07]]]
[[[ 0.0000000e+00  2.4214387e-08  3.3527613e-08 ... -2.9802322e-08
    2.9802322e-08  1.4901161e-08]
  [ 2.3469329e-07 -7.1525574e-07  5.9604645e-07 ... -1.7881393e-07
   -5.9604645e-07 -2.3841858e-07]
  [ 5.3644180e-07  1.1920929e-07  1.1920929e-07 ... -1.0728836e-06
   -2.9802322e-07 -6.5565109e-07]
  ...
  [ 1.7881393e-07 -4.7683716e-07  2.3841858e-07 ...  1.1920929e-07
   -8.3446